In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

In [2]:
def get_filepath_wtout_ext(file_wt_full_path):
    #/a/b/c.csv => /a/b/c
    return os.path.splitext(file_wt_full_path)[0]

In [3]:
def get_filename(file_wt_full_path):
    filepath_wout_ext = get_filepath_wtout_ext(file_wt_full_path)
    #/a/b/c => c
    return ntpath.basename(filepath_wout_ext)


In [4]:
def get_filepath(file_wt_full_path):
    #/a/b/c.csv => /a/b
    return os.path.dirname(file_wt_full_path)

In [5]:
def get_file(file_wt_full_path):
    #/a/b/c.csv => c.csv
    return os.path.basename(file_wt_full_path)

#### Things to note about the Answers
- They have value of 2 in the PostTypeId column
- They also have values in the ParentId column
    - which referes to the PostId of the Question

## Enter full path to CSV file

In [6]:
## Enter full path to CSV file
#xml_file = 'Posts*.csv'
common_path = '../../../../stackexchange_v2/workspace/input'
#common_path = '../SO_data_dump'

## Enter Relative Path to csv file
relative_path_posts = 'posts/Posts*.csv'
relative_path_javaposts = 'javaposts_csv/JavaPosts*.csv'

filepath_posts = '{}/{}'.format(common_path, relative_path_posts) # Post
filepath_javaposts = '{}/{}'.format(common_path, relative_path_javaposts) # JavaPost

In [7]:
#filepath=input()

## Read Post and JavaPost csv file

In [8]:
ddf_posts = dd.read_csv(filepath_posts, engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)

## Explore Post and Java Posts

In [9]:
ddf_posts

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,CommunityOwnedDate,ContentLicense,ParentId,DeletionDate,OwnerDisplayName,ClosedDate
npartitions=9195,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
ddf_javaposts = dd.read_csv(filepath_javaposts, engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)

In [11]:
ddf_javaposts

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,CommunityOwnedDate,ContentLicense,ParentId,DeletionDate,OwnerDisplayName,ClosedDate
npartitions=9195,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
ddf_posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId',
       'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title',
       'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
       'CommunityOwnedDate', 'ContentLicense', 'ParentId', 'DeletionDate',
       'OwnerDisplayName', 'ClosedDate'],
      dtype='object')

In [13]:
ddf_javaposts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId',
       'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title',
       'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
       'CommunityOwnedDate', 'ContentLicense', 'ParentId', 'DeletionDate',
       'OwnerDisplayName', 'ClosedDate'],
      dtype='object')

In [14]:
ddf_posts.npartitions

9195

In [15]:
ddf_javaposts.npartitions

9195

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [16]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="16GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

In [17]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=100, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.140:36058 Dashboard: http://192.168.94.140:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Process the Posts

## Total number of posts

In [18]:
#init_posts_len = ddf_posts.index.shape[0].compute() # compute length of posts_df
#print('Total number of posts: {}'.format(init_posts_len))

## Drop posts that have NaN in PostTypeId column
- because we are goint to make use of the PostTypeId in our comparison

In [19]:
#ddf_posts = ddf_posts[~ddf_posts.PostTypeId.isna()] 

## Total number of posts without NaN values in PostTypeId column

In [20]:
#posts_drp_ptid_nan_len = ddf_posts.index.shape[0].compute()
#print("Total number of posts without NaN values in the PostTypeId column: {}".format(posts_drp_ptid_nan_len))

## Drop posts that have NaN in the ParentId column
- because we are going to make use of the ParentId in our comparison

In [21]:
#ddf_posts = ddf_posts[~ddf_posts.ParentId.isna()]

In [22]:
#posts_drp_ptid_nan_len = ddf_posts.index.shape[0].compute()
#print("Total number of posts without NaN values in the ParentId column: {}".format(posts_drp_ptid_nan_len))

# Process the Java Posts

## Total number of javaposts
- also known as the Java Questions

In [23]:
#init_javaposts_len = ddf_javaposts.index.shape[0].compute()
#print("Total number of javaposts: {}".format(init_javaposts_len))

Total number of javaposts: 1796493


## Get all the java Answers
- PostId of Post must match with PostId of JavaPost

In [24]:
ddf_javaanswers = ddf_posts[ddf_posts.ParentId.isin(ddf_javaposts.Id.compute())] # get all the Posts Ids that is in JavaPosts

## Get the total number of java answers

In [ ]:
javaanswers_len = ddf_javaanswers.index.shape[0].compute() # compute length of javaanswers_df

## Total number of java answers

In [26]:
print("Total number of java answers: {}".format(javaanswers_len))

Total number of java answers: 2922551


## Make a folder in that directory

In [27]:
## Make a folder in that directory
folder = '{}/javaanswers_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

## Save files in that directory

In [28]:
## Save files in that directory
filename = 'JavaAnswers'
file = '{}/{}*.csv'.format(folder, filename)
_ = ddf_javaanswers.to_csv(file, sep=',', index=False)